## Prepare and read data

In [1]:
import gc
import itertools
from pprint import pprint
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import optuna.integration.lightgbm as lgb
import lightgbm as normal_lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Preprocess

In [4]:
def preprocess(train_df, test_df):
    
    ###########################
    # Functions of preprocess
    ###########################
    def get_multi_cat_cols(train_df):
        tmp = train_df.iloc[:1000]
        multi_cols = []
        for c in train_df.columns:
            sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
            if sep_num > 10:
                multi_cols.append(c)
        return multi_cols

    def add_rank_feature(df):
        rank_prefix_list = [
            "AssessBenefits",
            "AssessJob",
            "JobContactPriorities",
            "JobEmailPriorities",
            "AdsPriorities",
        ]
        for prefix in tqdm(rank_prefix_list):
            rank_cols = [c for c in df.columns if prefix in c]
            col_pairs = itertools.combinations(rank_cols, 2)
            for col_a, col_b in col_pairs:
                df[f"rank_diff_{prefix}_{col_a}_{col_b}"] = (df[col_a] - df[col_b]) / np.log2(df[[col_a, col_b]].max(axis=1))
        return df

    def get_basic_importance_cols(use_num=50):
        # basicなモデルのimportanceを読み込み
        importance_df = pd.read_csv("../data/importance/003_importance.csv")
        imp_feats = importance_df["feature"].iloc[:use_num].tolist()
        return imp_feats

    def make_agg_feature(train_df, test_df):
        imp_feats = get_basic_importance_cols(use_num=50)
        imp_cat_cols = [c for c in cat_cols if c in imp_feats] + non_basic_cat_cols
        imp_nume_cols = [c for c in nume_cols if c in imp_feats] + non_basic_nume_cols
        print(f"use cat col: {len(imp_cat_cols)}  nume col: {len(imp_nume_cols)}")
        #imp_nume_cols += [c for c in train_df.columns if c[:8] == "sum_answ"]
        all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
        for cat_col in tqdm(imp_cat_cols):
            for nume_col in imp_nume_cols:
                # one-hotは同じカテゴリの場合がある
                if cat_col == nume_col:
                    continue
                all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
                    all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
                all_df[f"agg_std_{cat_col}_{nume_col}"] = \
                    all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
                all_df[f"agg_max_{cat_col}_{nume_col}"] = \
                    all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
                all_df[f"agg_min_{cat_col}_{nume_col}"] = \
                    all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
                all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
                    all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
                all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
                    all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
        train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
        test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
        del all_df
        gc.collect()
        return train_df, test_df

    @feature_cache(reset=False)
    def target_encoding(train_df, test_df):
        te_cols = [c for c in train_df.columns if c in cat_cols]
        for c in tqdm(te_cols):
            new_col = "te_" + c
            train_df[new_col] = 0
            test_df[new_col] = 0
            for trn_idx, val_idx in fold_idx_list:
                mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
                train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
                test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
            train_df[new_col] = train_df[new_col].astype(np.float32)
            test_df[new_col] = test_df[new_col].astype(np.float32)
        return train_df, test_df

    @feature_cache(reset=False)
    def multiple_target_encoding(train_df, test_df):
         # multiple target encoding
        multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
        imp_feats = get_basic_importance_cols(use_num=30)
        multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
        combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

        for col_a, col_b in tqdm(combi_multi_te_cols):
            new_col = "te_" + col_a + "__" + col_b
            train_df[new_col] = 0
            test_df[new_col] = 0
            train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
            test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
            for trn_idx, val_idx in fold_idx_list:
                mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
                train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
                test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
            train_df[new_col] = train_df[new_col].astype(np.float32)
            test_df[new_col] = test_df[new_col].astype(np.float32)
            del train_df["tmp"], test_df["tmp"]
            gc.collect()
        return train_df, test_df   
    
    ################################
    # Columns infomation
    ################################
    original_cols = train_df.columns.tolist()
    multi_cat_cols = get_multi_cat_cols(train_df)

    nume_cols = [
        c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
        if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
    ]

    cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

    non_basic_nume_cols = []
    non_basic_cat_cols = []

    ################################
    #  Make feature
    ################################    
    
    # rank feature
    train_df = add_rank_feature(train_df)
    test_df = add_rank_feature(test_df)

    # multi -category encoding 
    for c in tqdm(multi_cat_cols):
        binarizer = MultiLabelBinarizer()
        train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
        test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
        train_arr = binarizer.fit_transform(train_multi_srs)
        test_arr = binarizer.transform(test_multi_srs)
        feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
        train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
        test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
        all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
        train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
        test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
        train_df = pd.concat([train_df, train_feat_df], axis=1)
        test_df = pd.concat([test_df, test_feat_df], axis=1)
        # ohe_featureはcategoryとnumerical両方として扱う
        nume_cols += feat_cols
        cat_cols += feat_cols
        # non_basic_nume_cols.append(f"sum_answer_{c}")
        # SVD
        svd = TruncatedSVD(n_components=2, random_state=2020)
        all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
        train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
        test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)
    
    # simple category encoding
    for c in cat_cols:
        train_df[c], uniques = pd.factorize(train_df[c], sort=True)
        test_df[c] = uniques.get_indexer(test_df[c])
    
    # reduce memory
    # numerical cols
    for c in nume_cols:
        if train_df[c].nunique() > 1000:
            train_df[c] = train_df[c].astype(np.float32)
            test_df[c] = test_df[c].astype(np.float32)
        else:
            train_df[c] = train_df[c].astype(np.float16)
            test_df[c] = test_df[c].astype(np.float16)
    # category cols
    for c in cat_cols:
        if train_df[c].max() > 32767:
            train_df[c] = train_df[c].astype(np.int32)
            test_df[c] = test_df[c].astype(np.int32)
        else:
            train_df[c] = train_df[c].astype(np.int16)
            test_df[c] = test_df[c].astype(np.int16)
    
    # change columns name
    train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
    test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]
   
    # aggregate feature
    train_df, test_df = make_agg_feature(train_df, test_df)

    # make train/validation index list for target encoding
    folds = KFold(n_splits=5, random_state=2020, shuffle=True)
    fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]
    
    # target encoding
    train_df, test_df = target_encoding(train_df, test_df)

    # multiple category target encoding 
    train_df, test_df = multiple_target_encoding(train_df, test_df)
    
    # make use columns list
    use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
    print(len(use_cols))
    
    return train_df, test_df, use_cols, fold_idx_list

## Training

In [5]:
def train(train_df, test_df, use_cols, fold_idx_list):

    lgb_params = {
                'objective': 'poisson',
                "metric": "rmse",
                "verbosity": -1,
                "boosting": "gbdt",
                'learning_rate': 0.05,
                'num_leaves': 64,
                'min_data_in_leaf': 80, 
                'max_depth': 4,
                "bagging_freq": 5,
                "bagging_fraction": 0.8,
                "lambda_l1": 0.5,
                "lambda_l2": 0.5,
                "feature_fraction": 0.1,
                "seed": 2020,
                "num_threads": -1,
                "max_bins": 30
    }
    def feature_selection(train_df, use_cols, n_features=1000):
        df = train_df.sample(30000, random_state=2020)
        train_dataset = normal_lgb.Dataset(
            df.loc[:, use_cols],
            label = df.loc[:, "Salary"]
        )
        model = normal_lgb.train(
                    lgb_params,
                    train_dataset,
                    2000,
                    valid_sets = [train_dataset],
                    verbose_eval=200,
                    early_stopping_rounds = None,
        )
        imp_df = pd.DataFrame()
        imp_df['feature'] = use_cols
        imp_df['gain'] = model.feature_importance(importance_type="gain")
        select_features = imp_df.sort_values(["gain"], ascending=False).iloc[: n_features]["feature"].tolist()
        return select_features
    
    importances = pd.DataFrame()
    oof_preds = np.zeros(len(train_df))
    models = []

    use_cols = feature_selection(train_df, use_cols, n_features=1000)

    # LightGBM のデータセット表現にする
    lgb_train = lgb.Dataset(train_df[use_cols], train_df["Salary"])

    # データセットの分割方法
    folds = KFold(n_splits=5, shuffle=True, random_state=42)

    # 最適化するときの条件
    lgbm_params = {
        'objective': 'poisson',
        'metric': 'rmse',
        'verbosity': -1,
    }
    # 基本的には cv() 関数のオプションがそのまま渡せる
    tuner_cv = lgb.LightGBMTunerCV(
        lgbm_params, lgb_train,
        num_boost_round=3000,
        early_stopping_rounds=300,
        verbose_eval=20,
        folds=folds,
    )
    # 最適なパラメータを探索する
    tuner_cv.run()

    # 最も良かったスコアとパラメータを書き出す
    print(f'Best score: {tuner_cv.best_score}')
    print('Best params:')
    pprint(tuner_cv.best_params)

## Prediction

In [6]:
def predict(test_df, models, use_cols, oof_score):
    test_pred = np.zeros(len(test_df))
    for model in models:
        test_pred += model.predict(test_df[use_cols]) / len(models)
    sub_df = pd.read_csv("../input/submit.csv")
    sub_df["Salary"] = test_pred
    sub_df.to_csv(f"../predict/{model_title}_{oof_score}.csv", index=False)

## EndPoint

In [7]:
model_title = "042_param_tuning"

In [8]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [9]:
train_df, test_df, use_cols, fold_idx_list = preprocess(train_df, test_df)


use cat col: 27  nume col: 10



2495


In [10]:
train(train_df, test_df, use_cols, fold_idx_list)

[200]	training's rmse: 20994.7
[400]	training's rmse: 19800.7
[600]	training's rmse: 19153.1
[800]	training's rmse: 18664.6
[1000]	training's rmse: 18255.4
[1200]	training's rmse: 17906.5
[1400]	training's rmse: 17552.9
[1600]	training's rmse: 17231
[1800]	training's rmse: 16939.5
[2000]	training's rmse: 16663.2


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[20]	cv_agg's rmse: 27099.4 + 280.393
[40]	cv_agg's rmse: 22970.6 + 302.526
[60]	cv_agg's rmse: 21860.2 + 316.68
[80]	cv_agg's rmse: 21408.7 + 317.47
[100]	cv_agg's rmse: 21154.2 + 312.451
[120]	cv_agg's rmse: 20994.6 + 301.832
[140]	cv_agg's rmse: 20881.9 + 291.75
[160]	cv_agg's rmse: 20793.1 + 277.285
[180]	cv_agg's rmse: 20727.5 + 266.31
[200]	cv_agg's rmse: 20668.5 + 264.407
[220]	cv_agg's rmse: 20629.4 + 258.415
[240]	cv_agg's rmse: 20595.3 + 258.822
[260]	cv_agg's rmse: 20568.9 + 261.778
[280]	cv_agg's rmse: 20549.5 + 258.121
[300]	cv_agg's rmse: 20534.6 + 258.881
[320]	cv_agg's rmse: 20519.5 + 255.372
[340]	cv_agg's rmse: 20508.7 + 256.038
[360]	cv_agg's rmse: 20506.4 + 261.563
[380]	cv_agg's rmse: 20499 + 262.401
[400]	cv_agg's rmse: 20496.7 + 259.69
[420]	cv_agg's rmse: 20494 + 257.801
[440]	cv_agg's rmse: 20494.2 + 261.308
[460]	cv_agg's rmse: 20497.4 + 257.27
[480]	cv_agg's rmse: 20497.7 + 256.892
[500]	cv_agg's rmse: 20495.1 + 256.888
[520]	cv_agg's rmse: 20495.5 + 255.777


feature_fraction, val_score: 20492.250527:  14%|#4        | 1/7 [02:52<17:12, 172.09s/it]

[20]	cv_agg's rmse: 27116.4 + 270.919
[40]	cv_agg's rmse: 22985.5 + 305.283
[60]	cv_agg's rmse: 21877.4 + 316.938
[80]	cv_agg's rmse: 21429.2 + 310.454
[100]	cv_agg's rmse: 21171.3 + 297.326
[120]	cv_agg's rmse: 21020 + 288.189
[140]	cv_agg's rmse: 20903.4 + 280.283
[160]	cv_agg's rmse: 20830.9 + 275.093
[180]	cv_agg's rmse: 20768.3 + 277.749
[200]	cv_agg's rmse: 20727.3 + 272.591
[220]	cv_agg's rmse: 20687.2 + 261.825
[240]	cv_agg's rmse: 20657 + 263.345
[260]	cv_agg's rmse: 20638.9 + 256.521
[280]	cv_agg's rmse: 20621.8 + 255.102
[300]	cv_agg's rmse: 20615.3 + 246.198
[320]	cv_agg's rmse: 20601.9 + 246.115
[340]	cv_agg's rmse: 20605.8 + 246.806
[360]	cv_agg's rmse: 20599.4 + 246.494
[380]	cv_agg's rmse: 20601.2 + 239.25
[400]	cv_agg's rmse: 20598.4 + 243.472
[420]	cv_agg's rmse: 20590.1 + 240.888
[440]	cv_agg's rmse: 20586.1 + 235.412
[460]	cv_agg's rmse: 20590.8 + 234.356
[480]	cv_agg's rmse: 20590.6 + 233.326
[500]	cv_agg's rmse: 20588.8 + 235.565
[520]	cv_agg's rmse: 20589.1 + 237

...

[20]	cv_agg's rmse: 27091.1 + 269.496
[40]	cv_agg's rmse: 22949.8 + 283.704
[60]	cv_agg's rmse: 21849.3 + 281.876
[80]	cv_agg's rmse: 21411 + 279.536
[100]	cv_agg's rmse: 21165.7 + 269.504
[120]	cv_agg's rmse: 21000.2 + 250.041
[140]	cv_agg's rmse: 20880.8 + 242.035
[160]	cv_agg's rmse: 20804.6 + 232.948
[180]	cv_agg's rmse: 20734.5 + 225.583
[200]	cv_agg's rmse: 20684.7 + 221.261
[220]	cv_agg's rmse: 20648.2 + 221.827
[240]	cv_agg's rmse: 20612.8 + 220.338
[260]	cv_agg's rmse: 20591.1 + 212.55
[280]	cv_agg's rmse: 20575 + 208.986
[300]	cv_agg's rmse: 20557.8 + 207.571
[320]	cv_agg's rmse: 20539.6 + 204.072
[340]	cv_agg's rmse: 20521.8 + 203.284
[360]	cv_agg's rmse: 20508.2 + 198.019
[380]	cv_agg's rmse: 20501.3 + 195.878
[400]	cv_agg's rmse: 20497.3 + 190.586
[420]	cv_agg's rmse: 20493.3 + 190.997
[440]	cv_agg's rmse: 20490.8 + 192.461
[460]	cv_agg's rmse: 20481.5 + 191.636
[480]	cv_agg's rmse: 20477.6 + 191.868
[500]	cv_agg's rmse: 20476.7 + 195.027
[520]	cv_agg's rmse: 20473.1 + 194

min_data_in_leaf, val_score: 20384.371698:  80%|########  | 4/5 [07:37<01:51, 111.83s/it]

[20]	cv_agg's rmse: 27094.3 + 273.067
[40]	cv_agg's rmse: 22937.4 + 300.766
[60]	cv_agg's rmse: 21798.7 + 286.489
[80]	cv_agg's rmse: 21329.6 + 283.314
[100]	cv_agg's rmse: 21075.6 + 279.926
[120]	cv_agg's rmse: 20912.6 + 263.317
[140]	cv_agg's rmse: 20800.4 + 260.682
[160]	cv_agg's rmse: 20711.5 + 255.18
[180]	cv_agg's rmse: 20652.9 + 239.837
[200]	cv_agg's rmse: 20599.3 + 235.672
[220]	cv_agg's rmse: 20558.6 + 240.093
[240]	cv_agg's rmse: 20526.4 + 238.491
[260]	cv_agg's rmse: 20494.3 + 235.645
[280]	cv_agg's rmse: 20471.3 + 240.424
[300]	cv_agg's rmse: 20455.3 + 242.79
[320]	cv_agg's rmse: 20434.3 + 245.568
[340]	cv_agg's rmse: 20419.5 + 243.622
[360]	cv_agg's rmse: 20411.1 + 238.462
[380]	cv_agg's rmse: 20403.8 + 237.035
[400]	cv_agg's rmse: 20395.8 + 242.889
[420]	cv_agg's rmse: 20388.9 + 236.114
[440]	cv_agg's rmse: 20384.5 + 231.001
[460]	cv_agg's rmse: 20379 + 227.009
[480]	cv_agg's rmse: 20372.1 + 228.253
[500]	cv_agg's rmse: 20367.8 + 227.282
[520]	cv_agg's rmse: 20364.6 + 22

min_data_in_leaf, val_score: 20358.731398: 100%|##########| 5/5 [09:34<00:00, 114.83s/it]


Best score: 20358.731398269807
Best params:
{'bagging_fraction': 0.8476866095748955,
 'bagging_freq': 3,
 'feature_fraction': 0.4,
 'lambda_l1': 2.8294406593458324e-08,
 'lambda_l2': 9.856795488014633,
 'metric': 'rmse',
 'min_child_samples': 100,
 'num_leaves': 31,
 'objective': 'poisson',
 'verbosity': -1}
